In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 8.1 계층적 색인
축에 대해 2개 이상의 색인index 단계를 지정  
= 높은 차원의 데이터를 낮은 차원의 형식으로 다룰 수 있도록 !  
  
  
__예시__

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a','a','a','b','b','c','c','d','d'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.399687
   2    0.047653
   3   -0.481830
b  1   -0.581706
   3   -0.022468
c  1    2.139005
   2   -0.918022
d  2    1.015559
   3   -1.403311
dtype: float64

* __Multi Index Series__  
→ 하위 계층으로 직접 접근 

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

→ 부분적 색인으로 접근 partial indexing

In [4]:
data['b']

1   -0.581706
3   -0.022468
dtype: float64

In [5]:
data['b':'c']

b  1   -0.581706
   3   -0.022468
c  1    2.139005
   2   -0.918022
dtype: float64

In [6]:
data.loc[['b','d']]

b  1   -0.581706
   3   -0.022468
d  2    1.015559
   3   -1.403311
dtype: float64

→ 하위 계층의 객체 선택

In [7]:
data.loc[:, 2]

a    0.047653
c   -0.918022
d    1.015559
dtype: float64

- __unstack__ : 데이터 새롭게 배열
- __stack__ : unstack 반대

In [8]:
data.unstack()

,1,2,3
a,0.399687,0.047653,-0.481830
b,-0.581706,NaN,-0.022468
c,2.139005,-0.918022,NaN
d,NaN,1.015559,-1.403311


In [9]:
data.unstack().stack()

a  1    0.399687
   2    0.047653
   3   -0.481830
b  1   -0.581706
   3   -0.022468
c  1    2.139005
   2   -0.918022
d  2    1.015559
   3   -1.403311
dtype: float64

* row & columns 모두 계층적 색인

In [10]:
frame = pd.DataFrame(np.arange(12).reshape(4,3),
                     index = [['a','a','b','b'], [1,2,1,2]],
                     columns = [['Ohio','Ohio','Colorado'],['Green','Red','Greedn']])
frame

Ohio     Colorado
    Green Red   Greedn
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

* 계층적 색인의 이름 이용

In [11]:
frame.index.names = ['key1', 'key2']

frame.columns.names = ['state','color']

frame

state      Ohio     Colorado
color     Green Red   Greedn
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### 8.1.1 계층 순서 바꾸고 정렬
> __swaplevel__('index1', 'index2') : 두 개 계층 번호/이름 뒤바뀐 새로운 객체 반환  
데이터는 변경되지 않음

In [12]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red   Greedn
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

> __sort_index__(level = n) : 단일 계층(n번째 계층)에 속한 데이터 정렬

In [13]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red   Greedn
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

* swaplevel 이용해 계층 바꾸기 → sort_index 이용해 사전적으로 정렬

In [14]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red   Greedn
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 8.1.2 계층별 요약 통계
> __level =__ 합을 구하고 싶은 단계

In [15]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red   Greedn
key2                    
1         6   8       10
2        12  14       16

In [16]:
frame.sum(level='color', axis=1)

color      Green  Red  Greedn
key1 key2                    
a    1         0    1       2
     2         3    4       5
b    1         6    7       8
     2         9   10      11

### 8.1.3 DataFrame의 column 사용

__예시__

In [17]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


> __set_index__ ( [ 'columnA' , 'columnB' ] ): 하나 이상의 column을 색인으로 하는 새로운 DataFrame 생성

In [18]:
frame2 = frame.set_index(['c','d'])

frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

> __drop = False__ :column 명시적으로 남겨두기

In [19]:
frame.set_index(['c','d'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

> __reset_index__ : 계층적 색인 단계가 column으로 이동  
set_index 반대

In [20]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


***

## 8.2 데이터 합치기

### 8.2.1 데이터베이스 스타일로 DataFrame 합치기
* __merge / join__ : 하나 이상의 __키__ 를 사용 → 데이터 집합의 row 합침  
  
__예시 - <u>다대일</u>__

In [21]:
df1 = pd.DataFrame({'key': ['b','b','a','c','a','a','b'],
                    'data1': range(7)})

df2 = pd.DataFrame({'key': ['a','b','d'],
                    'data2': range(3)})

In [22]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [23]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


> __merge__ 함수 호출 : 자동적으로 key = 중복된 column 이름

In [24]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


> key 지정  

* __on =__ 'key로 지정할 column'

In [25]:
pd.merge(df1, df2, on = 'key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


* 중복된 column 이름이 없는 경우, 따로 지정

In [26]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})

pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


\- merge 함수는 자동적으로 교집합을 반환 → 'c', 'd'에 해당하는 값이 빠짐
> __how =__ 'left' / 'right' / 'outer' : 왼쪽 / 오른쪽 / 외부 조인 수행
* 외부 조인: 합집합 결과 반환
* 왼쪽 / 오른쪽 조인 : 왼쪽 / 오른쪽의 모든 row 포함하는 결과 반환

In [27]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


__예시 - <u>다대다</u>__

In [28]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})

In [29]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [30]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


> merge

In [31]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [32]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


> 여러개의 키 병합 : on = [ ' column 이름이 담긴 리스트 ' ]

In [33]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


> 겹치는 column 이름을 변경 : __suffixes =__ ( 'suffix_1', 'suffix_2' )

In [34]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [35]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 8.2.2 색인 병합
병합하려는 키가 __DataFrame의 색인일때__
> __left_index = True / right_index = True__ (혹은 둘 다)

In [36]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [37]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [38]:
right1

,group_val
a,3.5
b,7.0


In [39]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


* 합집합 : 외부 조인 실행 __how = ' outer '__

In [40]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


__계층 색인 데이터__ 의 병합 
* 리스트로 여러개 column 지정 후 병합 
* 중복되는 색인값 : how = 'outer'

In [41]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [42]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [43]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [44]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [45]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True
         , how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


* 양쪽에 공통적으로 존재하는 여러 색인 병합

In [46]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [47]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [48]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [49]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


> __join__ : column 겹치지 않으면서 같거나 비슷한 인덱스를 지닌 여러 DataFrame 병합

In [51]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


* DataFrame의 column 중 하나에 대해 조인 수행  
__on =__ 'column 이름'

In [52]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


__색인 대 색인__ 으로 두 DataFrame 병합 : __join [__ 병합하려는 리스트 __]__  
  
__예시__

In [53]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [54]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [55]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### 8.2.3 축 따라 이어붙이기 (연결 binding, 적층 stacking)
> NumPy __concatenate__ : ndarray 이어붙임

In [56]:
arr = np.arange(12).reshape((3,4))

arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [57]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

> Pandas __concat__  
\- default: axis=0  
\- 새로운 Series 객체 생성

In [58]:
s1 = pd.Series([0,1], index=['a','b'])
s2 = pd.Series([2,3,4], index=['c','d','e'])
s3 = pd.Series([5,6], index=['f','g'])

pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

> concat __( axis = 1)__ : DataFrame 반환

In [59]:
pd.concat([s1, s2, s3], axis=1)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


> 교집합: __join = 'inner'__

In [60]:
s4 = pd.concat([s1, s3])

s4

a    0
b    1
f    5
g    6
dtype: int64

In [61]:
pd.concat([s1, s4], axis=1)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [62]:
pd.concat([s1,s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


> 병합하려는 축 직접 지정 : __join_axes__ = [ [ ' 축 이름 ' ] ]

In [63]:
pd.concat([s1, s4], axis=1, join_axes=[['a','c','b','e']])

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


> __이어 붙이기 전 개별 Series 구분__ : 이어 붙인 축에 대ㅐ해 계층적 색인 생성  
__keys =__

In [64]:
result = pd.concat([s1,s1,s3], keys=['one','two','three'])

result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [65]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


> Series를 axis = 1 로 병합 : keys = DataFrame의 column 제목

In [66]:
pd.concat([s1,s2,s3], axis=1, keys=['one','two','three'])

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


__DataFrame에 적용__

In [68]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index=['a','b','c'],
                   columns=['one','two'])

df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2), index=['a','c'],
                   columns=['three','four'])

In [69]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [70]:
df2

,three,four
a,5,6
c,7,8


In [71]:
pd.concat([df1,df2], axis=1, keys=['level1','level2'])

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

* 리스트 대신 __객체의 사전__ 을 넘기면: keys 옵션 = 사전의 키

In [72]:
pd.concat({'level1':df1, 'level2':df2}, axis=1)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

> __names = [__ ' 새로 ' , ' 생성될 ' , ' 계층의 ' , ' 이름 ' __]__

In [73]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

> __ignore_index = True__ : DataFrame의 row 색인이 분석에 필요한 데이터 포함하지 않을 때

In [74]:
df1 = pd.DataFrame(np.random.randn(3,4), columns=['a','b','c','d'])

df2 = pd.DataFrame(np.random.randn(2,3), columns=['b','d','a'])

In [75]:
df1

,a,b,c,d
0,1.418869,1.051161,-0.405437,-0.006970
1,1.797659,-0.742467,-0.981795,-1.759125
2,0.607487,-1.100252,1.209789,0.645724


In [76]:
df2

,b,d,a
0,-1.819313,-0.456278,-1.925135
1,-0.355476,-2.604213,0.560081


In [77]:
pd.concat([df1, df2], ignore_index=True)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,1.418869,1.051161,-0.405437,-0.006970
1,1.797659,-0.742467,-0.981795,-1.759125
2,0.607487,-1.100252,1.209789,0.645724
3,-1.925135,-1.819313,NaN,-0.456278
4,0.560081,-0.355476,NaN,-2.604213


### 8.2.4 겹치는 데이터 합치기
> 두 데이터셋의 색인의 일부 / 전체가 겹칠때  
NumPy  __where__

In [78]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan

In [79]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [80]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [81]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

> Series의 __combine_first__ : 위와 동일 + 정렬
  
  (호출하는 객체) . combine_first (누락된 데이터를 채울 인자)

In [82]:
b[:-2].combine_first(a[2:])   # b의 해당 위치에 a의 값 넣기

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

* column에 적용

In [83]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [84]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [85]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [87]:
df1.combine_first(df2)   # df1 의 NaN 값에 df2의 값 넣어서 합치기

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


***
## 8.3 재형성과 피벗
표 형식의 데이터를 재배치 하는 연산  
  
### 8.3.1 계층적 색인으로 재형성하기
* stack : 데이터의 column → row 피벗/회전
* unstack : row → column 피벗

In [88]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


> __stack__ : column → row 피벗. Series 객체 반환

In [90]:
result = data.stack()

result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

> __unstack__

In [91]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


* __끄집어낼 단계 지정__ : 레벨 숫자 / 이름 전달  
default : 가장 안쪽 레벨부터

In [92]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [93]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


* 해당 레벨의 모든 값이 하위 그룹에 속하지 않을 경우: unstack 시 누락된 데이터 발생 

In [94]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

data2 = pd.concat([s1, s2], keys=['one', 'two'])

data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [95]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


* stack 메서드는 누락 데이터를 자동으로 걸러냄 (연산 원상 복구 가능)

In [96]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [97]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [98]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

* unstack 레벨 = 결과의 가장 낮은 단계

In [99]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [100]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

* __축의 이름 지정__ 해서 stack

In [101]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### 8.3.2 긴 형식에서 넓은 형식으로 피벗하기
__긴 형식__ : 여러 시계열 / 둘 이상의 키를 가지고 있는 다른 관측 데이터에 사용  
* 시간 순서 나열  
  
__시계열 데이터 예제__

In [113]:
data = pd.read_csv('examples/macrodata.csv')

data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [114]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

data = data.reindex(columns=columns)
#data.index = periods.to_timestamp('D', 'end')
data.index = periods.to_timestamp()

ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [115]:
ldata[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


\- 각 row : 단일 관측치

> __pivot__ (' 인자1 ' , ' 인자2 ' , ' 인자3 ' , ' 인자4 '): 하나의 DataFrame에 'date' column의 시간값으로 색인된 __개별 item을 column으로 포함__
* 인자 1,2 : row, column 색인으로 사용될 column 이름
* 인자 3,4 : 채워 넣을 값

In [117]:
pivoted = ldata.pivot('date','item','value')  # 인자 1:'date'. 인자 2:'item'. 인자 3: value'

pivoted

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2
...,...,...,...
2008-07-01,-3.16,13324.600,6.0
2008-10-01,-8.79,13141.920,6.9
2009-01-01,0.94,12925.410,8.1


* 한 번에 2개 column 동시에 변형

In [118]:
ldata['value2'] = np.random.randn(len(ldata))

ldata[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,-0.241673
1,1959-01-01,infl,0.000,-0.132107
2,1959-01-01,unemp,5.800,-0.513079
3,1959-04-01,realgdp,2778.801,-0.066792
4,1959-04-01,infl,2.340,0.002552
5,1959-04-01,unemp,5.100,1.159585
6,1959-07-01,realgdp,2775.488,0.283851
7,1959-07-01,infl,2.740,0.668609
8,1959-07-01,unemp,5.300,0.049394
9,1959-10-01,realgdp,2785.204,1.028524


* 계층적 column 가지는 DataFrame 얻기 : 마지막 인자 생략

In [119]:
pivoted = ldata.pivot('date','item')

pivoted[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8 -0.132107 -0.241673 -0.513079
1959-04-01  2.34  2778.801   5.1  0.002552 -0.066792  1.159585
1959-07-01  2.74  2775.488   5.3  0.668609  0.283851  0.049394
1959-10-01  0.27  2785.204   5.6 -0.240200  1.028524 -0.075842
1960-01-01  2.31  2847.699   5.2 -0.190794  0.346272 -1.339831

In [120]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


* pivot = set_index( ) + unstack()

In [122]:
unstacked = ldata.set_index(['date','item']).unstack('item')

unstacked[:7]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8 -0.132107 -0.241673 -0.513079
1959-04-01  2.34  2778.801   5.1  0.002552 -0.066792  1.159585
1959-07-01  2.74  2775.488   5.3  0.668609  0.283851  0.049394
1959-10-01  0.27  2785.204   5.6 -0.240200  1.028524 -0.075842
1960-01-01  2.31  2847.699   5.2 -0.190794  0.346272 -1.339831
1960-04-01  0.14  2834.390   5.2 -0.509247 -3.030770 -0.916696
1960-07-01  2.70  2839.022   5.6 -0.109992  1.051304 -0.904742

### 8.3.3 넓은 형식에서 긴 형식으로 피벗
> pandas.__melt__ ( DataFrame, __[ '__ 그룹 구분자 __' ]__ ) : pivot 반대  
* 여러 column을 하나로 병합
* 입력보다 긴 형태로 DataFrame 반환

In [123]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [124]:
melted = pd.melt(df, ['key'])

melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


* 원래대로 되돌리기 : pivot 사용

In [125]:
reshaped = melted.pivot('key', 'variable', 'value')

reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


\- row 라벨로 사용하던 column에서 색인 생성  
  
 __→ reset_index__ :데이터를 다시 column 으로

In [126]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


* 데이터값으로 사용할 column들의 집합 지정  
__value_vars =__ [ '사용할', '칼럼', '이름' ]

In [127]:
pd.melt(df, id_vars = ['key'], value_vars = ['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [128]:
pd.melt(df, value_vars = ['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [129]:
pd.melt(df, value_vars = ['key','A','B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
